In [ ]:
import  torch
torch.cuda.is_available()

In [ ]:
!pip install transformers  torchvision pillow

In [ ]:
from transformers import AutoImageProcessor, ViTForImageClassification
import torch
from PIL import Image
import requests
torch.cuda.is_available()

In [ ]:
model_name = "WinKawaks/vit-tiny-patch16-224"


processor = AutoImageProcessor.from_pretrained(model_name)
model_test = ViTForImageClassification.from_pretrained(model_name)

In [ ]:
processor

In [ ]:
model_test

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
url = "https://huggingface.co/datasets/mishig/sample_images/resolve/main/tiger.jpg"
image = Image.open(requests.get(url, stream=True).raw)
plt.imshow(image)

inputs = processor(images=image, return_tensors="pt")

In [ ]:
inputs

In [ ]:
with torch.no_grad():
    outputs = model_test(**inputs)

logits = outputs.logits
predicted_class_idx = logits.argmax(-1).item()

print(model_test.config.id2label[predicted_class_idx])

In [ ]:
import torch.nn.functional as F

probs = F.softmax(logits, dim=-1)

# probs
topk = torch.topk(probs, k=5)
# topk
for score, idx in zip(topk.values[0], topk.indices[0]):
    print(model_test.config.id2label[idx.item()], score.item())

In [ ]:
model_test.config

In [ ]:
!pip install kaggle

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# https://www.kaggle.com/datasets/emmarex/plantdisease
!kaggle datasets download -d emmarex/plantdisease

In [ ]:
!unzip /content/plantdisease.zip

In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import os

# 1. Define Transformation (Preprocessing)
# We resize to 224x224 and normalize based on ImageNet standards
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# 2. Load Dataset from Folders
data_dir = '/content/PlantVillage' # Update this path
full_dataset = datasets.ImageFolder(root=data_dir, transform=data_transforms)


In [ ]:
full_dataset.classes

In [ ]:
# Get class names from subfolder names
class_names = full_dataset.classes
print(f"Detected {len(class_names)} classes: {class_names}")

# 3. Split into Train (80%) and Val (20%)
train_size = int(0.9 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_data, val_data = random_split(full_dataset, [train_size, val_size])

# 4. Create Data Loaders (The "Smart" part for batching)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True, num_workers=2)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False, num_workers=2)

In [ ]:
len(train_data), len(val_data)

In [ ]:
next(iter(train_loader))

In [ ]:
!pip install  peft datasets accelerate

In [ ]:
NUM_CLASSES = len(class_names)

model = ViTForImageClassification.from_pretrained(
    model_name,
    ignore_mismatched_sizes=True,
    num_labels=NUM_CLASSES
)

In [ ]:
model

Configure LoRa

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["query", "value"],
    lora_dropout=0.1,
    bias="none",
)

model = get_peft_model(model, config)

model.print_trainable_parameters()

In [ ]:
config

In [ ]:
for name, module in model.named_modules():
    print(name)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
model.save_pretrained("/content/drive/MyDrive/DL project/LoRA_ViT-V2")

In [ ]:
# best_loss = float('inf')

# for epoch in range(10):
#     epoch_loss = 0
#     for images, labels in train_loader:
#         outputs = model(pixel_values=images, labels=labels)
#         loss = outputs.loss

#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#         epoch_loss += loss.item()

#     avg_loss = epoch_loss / len(loader)
#     print(f"Epoch {epoch} Average Loss: {avg_loss}")

#     if avg_loss < best_loss:
#         best_loss = avg_loss
#         model.save_pretrained("best_vit_lora_model")
#         print("best model saved")

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
model.to(device)

In [ ]:
import torch.optim as optim
optimizer = optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=5e-5)

In [ ]:
# Initialize arrays to store losses
train_losses = []
val_losses = []
best_val_loss = float('inf')

for epoch in range(10):
    # --- TRAINING PHASE ---
    model.train()
    total_train_loss = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(pixel_values=images, labels=labels)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_train_loss += loss.item()

    avg_train_loss = total_train_loss / len(train_loader)
    train_losses.append(avg_train_loss)

    # --- VALIDATION PHASE ---
    model.eval()
    total_val_loss = 0

    with torch.no_grad(): # Disable gradient calculation for efficiency
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(pixel_values=images, labels=labels)
            val_loss = outputs.loss
            total_val_loss += val_loss.item()

    avg_val_loss = total_val_loss / len(val_loader)
    val_losses.append(avg_val_loss)

    print(f"Epoch {epoch}: Train Loss = {avg_train_loss:.4f} | Val Loss = {avg_val_loss:.4f}")

    # --- SAVE BEST MODEL BASED ON VALIDATION LOSS ---
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        model.save_pretrained("/content/drive/MyDrive/DL project/LoRA_ViT-V2/bestModel")
        print(" Best model saved (Validation loss improved)")

In [ ]:
# import matplotlib.pyplot as plt
# from PIL import Image
# import torch.nn.functional as F
from peft import PeftModel

base_model = ViTForImageClassification.from_pretrained(model_name ,     num_labels=len(class_names) ,     ignore_mismatched_sizes=True)

model_inference = PeftModel.from_pretrained(base_model, "/content/drive/MyDrive/DL project/LoRA_ViT-V2/bestModel")
model_inference.to(device)
model_inference.eval()

def infer_and_display(image_path, model, processor, class_names, device):
    # Load the image
    image = Image.open(image_path).convert("RGB")

    # Preprocess the image
    inputs = processor(images=image, return_tensors="pt").to(device)

    # Perform inference
    model.eval() # Set model to evaluation mode
    with torch.no_grad():
        outputs = model(**inputs)

    # Get logits and probabilities
    logits = outputs.logits
    probs = F.softmax(logits, dim=-1)

    # Get the top predicted class and confidence
    top_prob, top_idx = torch.topk(probs, k=1)
    predicted_class = class_names[top_idx.item()]
    confidence = top_prob.item()

    # Display the image
    plt.imshow(image)
    plt.title(f"Predicted: {predicted_class} (Confidence: {confidence:.2f})")
    plt.axis('off')
    plt.show()

    print(f"Predicted Class: {predicted_class}")
    print(f"Confidence: {confidence:.2f}")

In [ ]:
# img = "/content/PlantVillage/Pepper__bell___healthy/05f73b75-e898-4752-9c9b-ae745994eb01___JR_HL 8801.JPG"
img = "/content/PlantVillage/Tomato_Bacterial_spot/00a7c269-3476-4d25-b744-44d6353cd921___GCREC_Bact.Sp 5807.JPG"
infer_and_display(img, model_inference, processor, class_names, device)

In [ ]:
img = "/content/PlantVillage/Pepper__bell___healthy/05f73b75-e898-4752-9c9b-ae745994eb01___JR_HL 8801.JPG"
infer_and_display(img, model_inference, processor, class_names, device)